In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split

from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

In [ ]:
data_dir = './local_test/small_train'
val_data_dir = './input/gender-classification-dataset'
print(os.listdir(data_dir))
classes = os.listdir(data_dir)
print(classes)

In [ ]:
male_train_files = os.listdir(data_dir + "/male")
print("No. of test examples for male:", len(male_train_files))
print(male_train_files[:5])

In [ ]:
dataset_train = ImageFolder(data_dir+"/", transform=ToTensor())

In [ ]:
im_count = 0
for image,_ in dataset_train:
    im_count += 1
    print(image.shape)
    if im_count == 10:
        break 

In [ ]:
dataset_train.classes

In [ ]:
import torchvision.transforms as transforms
transform = transforms.Compose([transforms.Resize((100, 100)), transforms.ToTensor()])

In [ ]:
dataset_train = ImageFolder(data_dir+"/", transform = transform)

In [ ]:
dataset_val = ImageFolder(val_data_dir+"/Validation", transform = transform)

In [ ]:
im_count = 0
for image,_ in dataset_train:
    im_count += 1
    print(image.shape)
    if im_count == 10:
        break 

In [ ]:
img, label = dataset_train[2500]
print(img.shape, label)

In [ ]:
def show_example(img,label):
    print('Label: ', classes[label], '('+str(label)+')')
    plt.imshow(img.permute(1, 2, 0))

In [ ]:
show_example(*dataset_train[1000])

In [ ]:
# random_seed = 0
# torch.manual_seed(random_seed)

In [ ]:
train_size = 6000
val_size = len(dataset_train) - train_size

data_train, data_val = random_split(dataset_train, [train_size, val_size])
len(data_train), len(data_val)

In [ ]:
from torch.utils.data.dataloader import DataLoader

batch_size = 128

In [ ]:
train_dl = DataLoader(data_train, batch_size, shuffle = True, num_workers= 4, pin_memory=True)
val_dl = DataLoader(data_val, batch_size*2, num_workers=4, pin_memory=True)
test_dl = DataLoader(dataset_val, batch_size*2, num_workers=4, pin_memory=True)

In [ ]:
from torchvision.utils import make_grid

def show_batch(dl):
    for img, label in dl:
        fig, ax = plt.subplots(figsize = (16, 12))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(img, nrow = 16).permute(1,2,0))
        break

In [ ]:
show_batch(train_dl)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
sample_model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    nn.MaxPool2d(2, 2)
)

In [ ]:
for images, labels in train_dl:
    print("images shape: ", images.shape)
    out = sample_model(images)
    print("out shape", out.shape)
    break

In [ ]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
class GenCnnModel(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3,stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Output size: 64 x 50 x 50
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3,stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Output size: 128 x 25 x 25
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3,stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Output size: 256 x 12 x 12
            
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3,stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Output size: 512 x 6 x 6
            
            nn.Flatten(),
            nn.Linear(512*6*6, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 2))
        
    def forward(self, xb):
        return self.network(xb)

In [ ]:
sec_model = GenCnnModel()
sec_model

In [ ]:
for images, labels in train_dl:
    print("images shape: ", images.shape)
    out = sec_model(images)
    print("out shape: ", out.shape)
    print("out[0]: ", out[0])
    break

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
device

In [ ]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
train_loader = DeviceDataLoader(train_dl, device)
val_loader = DeviceDataLoader(val_dl, device)
test_loader = DeviceDataLoader(test_dl, device)
to_device(sec_model, device);

In [ ]:
@torch.no_grad()
def evaluate(sec_model, val_loader):
    sec_model.eval()
    outputs = [sec_model.validation_step(batch) for batch in val_loader]
    return sec_model.validation_epoch_end(outputs)

def fit(epochs, lr, sec_model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(sec_model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        sec_model.train()
        train_losses = []
        for batch in train_loader:
            loss = sec_model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(sec_model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        sec_model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
sec_model = to_device(GenCnnModel(), device)

In [ ]:
evaluate(sec_model, val_loader)

In [ ]:
num_epochs = 5
opt_func = torch.optim.Adam
lr = 0.001

In [ ]:
history = fit(num_epochs, lr, sec_model, train_loader, val_loader, opt_func)

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');

In [ ]:
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

In [ ]:
plot_accuracies(history)

In [ ]:
plot_losses(history)

In [ ]:
result = evaluate(sec_model, test_loader)
result

In [ ]:
torch.save(sec_model.state_dict(), 'local_model.pth')